In [1]:
import pandas as pd
import numpy as np
from sklearn.isotonic import IsotonicRegression
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
for year in range(2003,2017):
    data = pd.read_csv('data/%d.csv' % (year,));
    if year==2003:
        df = data.loc[0:1,:]
    df = df.append(data.iloc[np.random.choice(range(len(data)),size=75000,replace=False)])

In [4]:
data.shape

(903478, 40)

In [8]:
data.columns

Index([u'Year', u'Tourn.#', u'Player_#', u'Course_#',
       u'Permanent_Tournament_#', u'Player_First_Name', u'Player_Last_Name',
       u'Round', u'Tournament_Name', u'Course_Name', u'Hole', u'Hole_Score',
       u'Par_Value', u'Yardage', u'Shot', u'Shot_Type(S/P/D)', u'#_of_Strokes',
       u'From_Location(Scorer)', u'From_Location(Enhanced)',
       u'To_Location(Scorer)', u'To_Location(Enhanced)', u'Distance',
       u'Distance_to_Pin', u'Around_the_Green_Flag', u'1st_Putt_Flag',
       u'Distance_to_Hole_after_the_Shot', u'Time', u'Elevation',
       u'Strokes_Gained/Baseline', u'Strokes_Gained_Category',
       u'Recovery_Shot', u'Shots_taken_from_location', u'Went_to_X',
       u'Went_to_Y', u'Went_to_Z', u'Started_at_X', u'Started_at_Y',
       u'Started_at_Z', u'Distance_from_hole', u'Strokes_Gained'],
      dtype='object')

In [18]:
df.shape

(1050002, 41)

In [27]:
def print_full(x):
    pd.set_option('display.max_columns', None)
    print(x)
    pd.reset_option('display.max_columns')

In [34]:
def convert_cats(cat,dist):
    if cat in ['Green Side Bunker','Fairway Bunker']:
        return 'Bunker'
    elif cat not in ['Green','Fairway','Fringe','Primary Rough','Intermediate Rough','Tee Box']:
        return 'Other'
    elif cat=='Fringe' and dist>120:
        return 'Intermediate Rough'
    else:
        return cat

In [35]:
df.insert(len(df.columns),'Cat',[convert_cats(c,d) for c,d in zip(df['From_Location(Scorer)'],df['Distance_from_hole'])])

In [38]:
df.Cat.value_counts()/len(df)

Green                 0.414115
Tee Box               0.255837
Fairway               0.169531
Primary Rough         0.079005
Bunker                0.035701
Intermediate Rough    0.023388
Fringe                0.017433
Other                 0.004990
Name: Cat, dtype: float64

In [48]:
cat_freq = pd.DataFrame({'Category':['green <5 feet','green 5-10 feet','green 10-20 feet','green 20+ feet',
                          'rough <30 yards','rough 30-125 yards','rough 125+ yards',
                          'fairway <100 yards','fairway 100-200 yards','fairway 200+ yards',
                          'bunker play','tee shots par threes','tee shots par fours and fives'],
              'Number_of_Shots':[len(df[(df.Cat=='Green') & (df.Distance_from_hole<5)]),
                                 len(df[(df.Cat=='Green') & (df.Distance_from_hole<10) & (df.Distance_from_hole>=5)]),
                                 len(df[(df.Cat=='Green') & (df.Distance_from_hole<20) & (df.Distance_from_hole>=10)]),
                                 len(df[(df.Cat=='Green') & (df.Distance_from_hole>=20)]),
                                 len(df[(df.Cat=='Primary Rough') & (df.Distance_from_hole<30*3)]),
                                 len(df[(df.Cat=='Primary Rough') & (df.Distance_from_hole>=30*3) & (df.Distance_from_hole<125*3)]),
                                 len(df[(df.Cat=='Primary Rough') & (df.Distance_from_hole>=125*3)]),
                                 len(df[(df.Cat=='Fairway') & (df.Distance_from_hole<100*3)]),
                                 len(df[(df.Cat=='Fairway') & (df.Distance_from_hole<200*3) & (df.Distance_from_hole>=100*3)]),
                                 len(df[(df.Cat=='Fairway') & (df.Distance_from_hole>=200*3)]),
                                 len(df[df.Cat=='Bunker']),
                                 len(df[(df.Cat=='Tee Box') & (df.Par_Value==3)]),
                                 len(df[(df.Cat=='Tee Box') & (df.Par_Value!=3)])]})

In [49]:
cat_freq['Frequency'] = cat_freq.Number_of_Shots/len(df)

In [50]:
cat_freq.sort_values('Frequency')

,Category,Number_of_Shots,Frequency
5,rough 30-125 yards,15552,0.014811
4,rough <30 yards,26212,0.024964
9,fairway 200+ yards,34139,0.032513
10,bunker play,37486,0.035701
6,rough 125+ yards,41191,0.039229
7,fairway <100 yards,45213,0.043060
11,tee shots par threes,61163,0.058250
1,green 5-10 feet,64925,0.061833
2,green 10-20 feet,71373,0.067974
3,green 20+ feet,88710,0.084486


In [51]:
cat_freq.Frequency.sum()

0.9541886586882691